In [10]:
import numpy as np
import pandas as pd
import psycopg2
import string
import geopy.distance
import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
conn = psycopg2.connect("dbname = soaring_predictor")


In [3]:
LOGS = pd.read_sql('SELECT * FROM flight_log;',conn)

In [15]:

# get altitude string

def get_alt(alt_str):
    if len(alt_str)<=4:
        return int(alt_str)
    else:
        alt_str1 = int(alt_str[0:5].strip('-'))
        alt_str2 = int(alt_str[5:10].strip('-'))
        if alt_str2 != 0:
            return alt_str2
        elif alt_str !=0:
            return alt_str1

# for debug purposes ony - returns list of strings
def find_alt_entry(igc_file):
    a = re.search('WA([-0123456789]{3,16})', igc_file)
    if a:
        return a.group(1)
    

# returns the max altitude in the fligh (in meters)

def max_alt(igc_file):
    m = re.findall('WA([-0123456789]{2,16})', igc_file)
    return max([get_alt(i) for i in m])
    
def list_alt(igc_file):
    m = re.findall('WA([-0123456789]{2,16})', igc_file)
    return [get_alt(i) for i in m]


# returns the total altitude gain in the flight (in meters):

# returns landing coordinates

def list_coord(igc_file):
    items = re.findall('B([01234567890NW]{23})A', igc_file )
    output = []
    for item in items:
        e = item.find('N')
        lat = item[e-7:e+1]
        lon = item[e+1:]
        lat = dec_lat(lat)
        lon = dec_lon(lon)
        output.append((lat,lon))
    return output  

def total_alt_gain(igc_file):
    total_alt_gain = 0
    m = re.findall('WA([-0123456789]{3,16})', igc_file)
    for s, t in zip(m, m[1:]):
        try:
            if len(t) <= 4:
                if int(t) > int(s):
                    total_alt_gain += (int(t) - int(s))
            elif int(t[5:10]) > int(s[5:10]):
                total_alt_gain += (int(t[5:10]) - int(s[5:10]))
        except:
            next
    return total_alt_gain

    
    

# returns the coordinates in decimals from string

def get_coord(item):
    lat = dec_lat(item[8:16])
    lon = dec_lon(item[16:25])
    return (lat,lon)

# returns the latitude in decimals from segregated string


def dec_lat(lat):
    D = float(lat[0:2])
    M = float(lat[2:7])
    S = float(lat[4:7])
    DD = D + M/60000 
    if lat[-1] == 'N':
        lat = DD
    elif lat[-1] == 'S':
        lat = -DD
    return lat

# returns the longitude in decimals from segregated string

def dec_lon(lon):
    D = float(lon[0:3])
    M = float(lon[3:8])
    S = float(lon[5:8])
    DD = D + M/60000 
    if lon[-1] == 'E':
        lon = DD
    elif lon[-1] == 'W':
        lon = -DD
    return lon


# returns the euclidian distance between landing and launch

def final_eucl_distance(igc_file):
    return geopy.distance.vincenty(launch_coord(igc_file), landing_coord(igc_file)).km


# returns the euclidian distance between two points (entry in coordinates, returns distance in km)

def eucl_distance(pt1,pt2):
    return geopy.distance.vincenty(pt1, pt2).km



In [17]:
list_alt = list_alt(LOGS['log'][2])
list_coord = list_coord(LOGS['log'][2])

In [19]:
print(len(list_coord))
print(len(list_alt))

5711
5711


In [32]:
def find_thermal(list_alt,climb=3,pace = 10,duration=60):
    thermals = []
    for idx,(item1,item2) in enumerate(zip(list_alt,list_alt[1:])):
            if item1 < item2:
                thermals.append(idx)
    return thermals
            
            
        

In [35]:
for i,item in enumerate(list_alt[0:30]):
    print(i,item)

0 505
1 505
2 506
3 507
4 507
5 507
6 506
7 506
8 506
9 506
10 505
11 504
12 503
13 502
14 503
15 504
16 504
17 503
18 501
19 499
20 499
21 500
22 502
23 503
24 503
25 504
26 504
27 505
28 508
29 511


In [65]:
gap = 10
thermals = find_thermal(list_alt)
idx_lis = []
for i,j in zip(thermals,thermals[1:]):
    idx_lis.append(j - i < gap)

In [70]:
thermals

[1,
 2,
 13,
 14,
 20,
 21,
 22,
 24,
 26,
 27,
 28,
 29,
 30,
 34,
 35,
 37,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 48,
 49,
 50,
 52,
 53,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 66,
 67,
 68,
 71,
 72,
 73,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 85,
 87,
 88,
 90,
 91,
 93,
 94,
 95,
 96,
 101,
 102,
 104,
 106,
 108,
 109,
 110,
 111,
 112,
 113,
 117,
 118,
 119,
 122,
 123,
 124,
 127,
 128,
 129,
 130,
 131,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 163,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 194,
 195,
 196,
 197,
 198,
 199,
 202,
 209,
 216,
 217,
 218,
 219,
 225,
 227,
 229,
 272,
 273,
 274,
 275,
 278,
 283,
 285,
 289,
 290,
 291,
 292,
 293,
 294,
 295,
 296,
 297,
 298,
 299,
 300,
 301,
 302,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 311,
 312,
 313,
 314,
 315,
 316,
 317,
 318,
 319,
 320,
 323,
 324,
 325,
 328,
 329,
 330

In [ ]:
from sklearn.ensemble import km 

In [47]:
thermals = np.array(thermals)
idx = np.array(idx_lis)

In [53]:
thermals[:-1][idx_lis]

array([   1,   13,   20, ..., 5576, 5643, 5696])

In [54]:
thermals

array([   1,    2,   13, ..., 5677, 5696, 5698])